In [1]:
import joblib
import pandas as pd
import numpy as np

try:
    kit_carregado = joblib.load('modelo_final_completo.pkl')
    print("Kit carregado com sucesso!")

    modelo = kit_carregado['modelo']
    escalonador = kit_carregado['escalonador']
    colunas_modelo = kit_carregado['lista_de_colunas']
    top_categorias = kit_carregado['top_categorias']
except FileNotFoundError:
    print("ERRO: O arquivo 'modelo_final_completo.pkl' não foi encontrado.")
    print("Certifique-se de que ele está na mesma pasta que este notebook.")

def prever_nota_filme(df_filme_raw):
    """
    Recebe um DataFrame com 1 linha de dados BRUTOS de um filme
    e retorna a previsão da nota do IMDB.
    """
    df_proc = df_filme_raw.copy()

    df_proc['Runtime'] = df_proc['Runtime'].str.replace(' min', '').astype(int)
    df_proc['Gross'] = df_proc['Gross'].str.replace(',', '').astype(float)
    df_proc['No_of_Votes_log'] = np.log10(df_proc['No_of_Votes'] + 1)
    df_proc['Gross_log'] = np.log10(df_proc['Gross'] + 1)
    df_proc = df_proc.drop(columns=['No_of_Votes', 'Gross'])

    genre_dummies = df_proc['Genre'].str.get_dummies(sep=', ')
    df_proc = pd.concat([df_proc, genre_dummies], axis=1)
    df_proc = df_proc.drop('Genre', axis=1)

    for col, top_cats in top_categorias.items():

        if col in df_proc.columns:
            df_proc[col] = df_proc[col].where(df_proc[col].isin(top_cats), 'Other')

    df_proc = pd.get_dummies(df_proc, drop_first=True)
    df_proc_final = df_proc.reindex(columns=colunas_modelo, fill_value=0)

    dados_escalonados = escalonador.transform(df_proc_final)
    previsao = modelo.predict(dados_escalonados)

    return previsao[0]

shawshank_data = {
    'Series_Title': 'The Shawshank Redemption', 'Released_Year': '1994',
    'Certificate': 'A', 'Runtime': '142 min', 'Genre': 'Drama',
    'Overview': 'Two imprisoned men bond over a number of years...',
    'Meta_score': 80.0, 'Director': 'Frank Darabont', 'Star1': 'Tim Robbins',
    'Star2': 'Morgan Freeman', 'Star3': 'Bob Gunton', 'Star4': 'William Sadler',
    'No_of_Votes': 2343110, 'Gross': '28,341,469'
}
filme_para_teste = pd.DataFrame([shawshank_data])

nota_prevista = prever_nota_filme(filme_para_teste)

print(f"A nota prevista para '{filme_para_teste['Series_Title'].iloc[0]}' é: {nota_prevista:.2f}")


Kit carregado com sucesso!
A nota prevista para 'The Shawshank Redemption' é: 7.93
